## Fine-tuning
Fine-tune models for better results and efficiency.
Fine-tuning lets you get more out of the models available through the API by providing:

- Higher quality results than prompting
- Ability to train on more examples than can fit in a prompt
- Token savings due to shorter prompts
- Lower latency requests

Using demonstrations to show how to perform a task is often called "few-shot learning."
Fine-tuning improves on __few-shot learning__ by training on many more examples than can fit in the prompt, letting you achieve better results on a wide number of tasks. Once a model has been fine-tuned, you won't need to provide as many examples in the prompt. This saves costs and enables lower-latency requests.

At a high level, fine-tuning involves the following steps:

1. Prepare and upload training data
2. Train a new fine-tuned model
3. Evaluate results and go back to step 1 if needed
4. Use your fine-tuned model


### Which models can be fine-tuned?
Fine-tuning is currently available for the following models:

- gpt-4o-2024-08-06
- gpt-4o-mini-2024-07-18
- gpt-4-0613
- gpt-3.5-turbo-0125
- gpt-3.5-turbo-1106
- gpt-3.5-turbo-0613

### When to use fine-tuning
Fine-tuning OpenAI text generation models can make them better for specific applications, but it requires a careful investment of time and effort. We recommend first attempting to get good results with prompt engineering, prompt chaining (breaking complex tasks into multiple prompts), and function calling

### Common use cases
Some common use cases where fine-tuning can improve results:

- Setting the style, tone, format, or other qualitative aspects
- Improving reliability at producing a desired output
- Correcting failures to follow complex prompts
- Handling many edge cases in specific ways
- Performing a new skill or task that’s hard to articulate in a prompt
  
One high-level way to think about these cases is when it’s easier to "show, not tell".


### Preparing your dataset
Each example in the dataset should be a conversation in the same format as our Chat Completions API, specifically a list of messages where each message has a role, content, and optional name.
### Example format

In [ ]:
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "Paris, as if everyone doesn't know that already."}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "Oh, just some guy named William Shakespeare. Ever heard of him?"}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "Around 384,400 kilometers. Give or take a few, like that really matters."}]}

### Multi-turn chat examples
Examples in the chat format can have multiple messages with the assistant role. The default behavior during fine-tuning is to train on all assistant messages within a single example. To skip fine-tuning on specific assistant messages, a weight key can be added disable fine-tuning on that message, allowing you to control which assistant messages are learned. The allowed values for weight are currently 0 or 1. Some examples using weight for the chat format are below.

In [ ]:
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "Paris", "weight": 0}, {"role": "user", "content": "Can you be more sarcastic?"}, {"role": "assistant", "content": "Paris, as if everyone doesn't know that already.", "weight": 1}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "William Shakespeare", "weight": 0}, {"role": "user", "content": "Can you be more sarcastic?"}, {"role": "assistant", "content": "Oh, just some guy named William Shakespeare. Ever heard of him?", "weight": 1}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "384,400 kilometers", "weight": 0}, {"role": "user", "content": "Can you be more sarcastic?"}, {"role": "assistant", "content": "Around 384,400 kilometers. Give or take a few, like that really matters.", "weight": 1}]}

### Example count recommendations
To fine-tune a model, you are required to provide at least 10 examples. We typically see clear improvements from fine-tuning on 50 to 100 training examples with gpt-4o-mini and gpt-3.5-turbo, but the right number varies greatly based on the exact use case.
We recommend starting with __50__ well-crafted demonstrations and seeing if the model shows signs of improvement after fine-tuning.

### Train and test splits
After collecting the initial dataset, we recommend splitting it into a training and test portion. When submitting a fine-tuning job with both training and test files, we will provide statistics on both during the course of training. These statistics will be your initial signal of how much the model is improving. Additionally, constructing a test set early on will be useful in making sure you are able to evaluate the model after training, by generating samples on the test set.

### Token limits
Token limits depend on the model you select. Here is an overview of the maximum inference context length and training examples context length for gpt-4o-mini and gpt-3.5-turbo models:


In [ ]:

Model	                Inference context length	Training examples context length
gpt-4o-2024-08-06	    128,000 tokens	            65,536 tokens (128k coming soon)
gpt-4o-mini-2024-07-18	128,000 tokens	            65,536 tokens (128k coming soon)
gpt-3.5-turbo-0125	    16,385 tokens	            16,385 tokens
gpt-3.5-turbo-1106	    16,385 tokens	            16,385 tokens
gpt-3.5-turbo-0613	    16,385 tokens	            4,096 tokens

Examples longer than the default will be truncated to the maximum context length which removes tokens from the end of the training example(s). To be sure that your entire training example fits in context, consider checking that the total token counts in the message contents are under the limit.

### Check data formatting
Once you have compiled a dataset and before you create a fine-tuning job, it is important to check the data formatting. To do this, we created a simple Python script which you can use to find potential errors, review token counts, and estimate the cost of a fine-tuning job.

### [Fine-tuning data format validation](https://cookbook.openai.com/examples/chat_finetuning_data_prep)

### Upload a training file
Once you have the data validated, the file needs to be uploaded using the Files API in order to be used with a fine-tuning jobs:

In [ ]:
# Create a fine-tuning job with DPO
#DPO stands for Direct Preference Optimization. It is a method used to fine-tune AI models based on human preferences without needing a reward model.
from openai import OpenAI

client = OpenAI()

job = client.fine_tuning.jobs.create(
    training_file="file-all-about-the-weather", #The training_file parameter is the file ID that was returned when the training file was uploaded to the OpenAI API
    model="gpt-4o-2024-08-06",
    method={
        "type": "dpo",
        "dpo": {
            "hyperparameters": {"beta": 0.1},
        },
    },
)

### DPO (Direct Preference Optimization)
DPO is a fine-tuning method that trains a model using human preference data without needing a separate reward model (unlike RLHF).

#### How it works?

- Humans rank two responses (which one is better).
- The model learns to prefer better responses over worse ones.
- It directly adjusts the model using these preferences without reinforcement learning.
#### Why use DPO?

- Faster than RLHF (no reward model needed).
- More stable (avoids complex RL training).

### SFT (Supervised Fine-Tuning)
SFT is basic fine-tuning where the model is trained on labeled data (input → correct output).

#### How it works?

- You give the model pairs of inputs and correct answers.
- It learns to mimic the labeled data.
- This is the first step before applying RLHF or DPO.
#### Why use SFT?

- It teaches the model good responses before preference tuning.
- Used for instruction tuning (e.g., making ChatGPT follow prompts better).

In DPO (Direct Preference Optimization), __beta__ is a hyperparameter that controls the trade-off between:

1. Staying close to the original model (before fine-tuning).
2. Adapting to human preferences (based on preference data).
#### What Does beta: 0.1 Mean?
The beta hyperparameter is a new option that is only available for DPO. It's a floating point number between 0 and 2 that controls how strictly the new model will adhere to its previous behavior, versus aligning with the provided preferences.
- A high number will be more conservative (favoring previous behavior)
- A lower number will be more aggressive (favor the newly provided preferences more often).

### RLHF (Reinforcement Learning from Human Feedback)
RLHF is a fine-tuning method where a model is trained using human feedback and reinforcement learning.

#### How RLHF Works?
- Supervised Fine-Tuning (SFT) → Train the model on labeled data (input → correct output).
- Preference Data Collection → Humans rank multiple responses from the model (which is better).
- Train a Reward Model (RM) → A new model learns to predict human preferences from ranked responses.
- Reinforcement Learning (RL) → The main model is fine-tuned using PPO (Proximal Policy Optimization) to maximize the reward model's score.
#### Why RLHF?
- Helps the model align with human values and avoid harmful responses.
- Used in ChatGPT, GPT-4, and other AI assistants.

### Size limits and large uploads
The maximum upload size is 512 MB using the Files API. You can upload files up to 8 GB in multiple parts using the Uploads API. We recommend starting small, as you don't need a lot of data to see improvements.

### Create a fine-tuned model
After ensuring you have the right amount and structure for your dataset, and have uploaded the file, the next step is to create a fine-tuning job. We support creating fine-tuning jobs via the fine-tuning UI or programmatically.

To start a fine-tuning job using the OpenAI SDK:

In [ ]:
#Create fine-tuning job
from openai import OpenAI
client = OpenAI()
client.file_tuning.jobs.create(
    training_file = "file-abc123", #The training_file parameter is the file ID that was returned when the training file was uploaded to the OpenAI API
    model = "gpt-4o-mini-2024-07-08"
)

In addition to creating a fine-tuning job, you can also list existing jobs, retrieve the status of a job, or cancel a job.

In [2]:
# Work with fine-tuning jobs
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve("ftjob-abc123")

# Cancel a job
client.fine_tuning.jobs.cancel("ftjob-abc123")

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-abc123", limit=10)

# Delete a fine-tuned model
client.models.delete("ft:gpt-3.5-turbo:acemeco:suffix:abc123")


### Use a fine-tuned model

In [ ]:
#Use fine-tuned model
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
    model="ft:gpt-4o-mini:my-org:custom_suffix:id",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello!"}
    ]
)

print(completion.choices[0].message)

### Use a checkpointed model
In addition to creating a final fine-tuned model at the end of each fine-tuning job, OpenAI will create one full model checkpoint for you at the end of each training epoch. These checkpoints are themselves full models that can be used within our completions and chat-completions endpoints. Checkpoints are useful as they potentially provide a version of your fine-tuned model from before it experienced overfitting.

To access these checkpoints,

1. Wait until a job succeeds, which you can verify by querying the status of a job.
2. Query the checkpoints endpoint with your fine-tuning job ID to access a list of model checkpoints for the fine-tuning job.
   
For each checkpoint object, you will see the fine_tuned_model_checkpoint field populated with the name of the model checkpoint. You may now use this model just like you would with the final fine-tuned model.

In [ ]:
{
    "object": "fine_tuning.job.checkpoint",
    "id": "ftckpt_zc4Q7MP6XxulcVzj4MZdwsAB",
    "created_at": 1519129973,
    "fine_tuned_model_checkpoint": "ft:gpt-3.5-turbo-0125:my-org:custom-suffix:96olL566:ckpt-step-2000",
    "metrics": {
        "full_valid_loss": 0.134,
        "full_valid_mean_token_accuracy": 0.874
    },
    "fine_tuning_job_id": "ftjob-abc123",
    "step_number": 2000
}

Each checkpoint will specify its:

- step_number: The step at which the checkpoint was created (where each epoch is number of steps in the training set divided by the batch size)
- metrics: an object containing the metrics for your fine-tuning job at the step when the checkpoint was created.

### Analyzing your fine-tuned model
We provide the following training metrics computed over the course of training:

- training loss
- training token accuracy
- valid loss
- valid token accuracy
  
Valid loss and valid token accuracy are computed in two different ways - on a small batch of the data during each step, and on the full valid split at the end of each epoch. The full valid loss and full valid token accuracy metrics are the most accurate metric tracking the overall performance of your model. These statistics are meant to provide a sanity check that training went smoothly (loss should decrease, token accuracy should increase). While an active fine-tuning jobs is running, you can view an event object which contains some useful metrics:

In [ ]:
{
    "object": "fine_tuning.job.event",
    "id": "ftevent-abc-123",
    "created_at": 1693582679,
    "level": "info",
    "message": "Step 300/300: training loss=0.15, validation loss=0.27, full validation loss=0.40",
    "data": {
        "step": 300,
        "train_loss": 0.14991648495197296,
        "valid_loss": 0.26569826706596045,
        "total_steps": 300,
        "full_valid_loss": 0.4032616495084362,
        "train_mean_token_accuracy": 0.9444444179534912,
        "valid_mean_token_accuracy": 0.9565217391304348,
        "full_valid_mean_token_accuracy": 0.9089635854341737
    },
    "type": "metrics"
}

After a fine-tuning job has finished, you can also see metrics around how the training process went by querying a fine-tuning job, extracting a file ID from the result_files, and then retrieving that files content. Each results CSV file has the following columns: step, train_loss, train_accuracy, valid_loss, and valid_mean_token_accuracy.

In [ ]:
step,train_loss,train_accuracy,valid_loss,valid_mean_token_accuracy
1,1.52347,0.0,,
2,0.57719,0.0,,
3,3.63525,0.0,,
4,1.72257,0.0,,
5,1.52379,0.0,,

You can set the hyperparameters as is shown below:

In [ ]:
#Setting hyperparameters
from openai import OpenAI
client = OpenAI()

client.fine_tuning.jobs.create(
    training_file="file-abc123",
    model="gpt-4o-mini-2024-07-18",
    method={
        "type": "supervised",
        "supervised": {
            "hyperparameters": {"n_epochs": 2},
        },
    },
)

### Vision fine-tuning
Fine-tuning is also possible with images in your JSONL files. Just as you can send one or many image inputs to chat completions, you can include those same message types within your training data. Images can be provided either as HTTP URLs or data URLs containing base64 encoded images.

Here's an example of an image message on a line of your JSONL file. Below, the JSON object is expanded for readibility, but typically this JSON would appear on a single line in your data file:

In [ ]:
{
  "messages": [
    { "role": "system", "content": "You are an assistant that identifies uncommon cheeses." },
    { "role": "user", "content": "What is this cheese?" },
    { "role": "user", "content": [
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/3/36/Danbo_Cheese.jpg"
          }
        }
      ]
    },
    { "role": "assistant", "content": "Danbo" }
  ]
}

### Image dataset requirements
#### Size
- Your training file can contain a maximum of 50,000 examples that contain images (not including text examples).
- Each example can have at most 10 images.
- Each image can be at most 10 MB.
#### Format
- Images must be JPEG, PNG, or WEBP format.
- Your images must be in the RGB or RGBA image mode.
- You cannot include images as output from messages with the assistant role.
#### Content moderation policy
We scan your images before training to ensure that they comply with our usage policy. This may introduce latency in file validation before fine-tuning begins.

Images containing the following will be excluded from your dataset and not used for training:

- People
- Faces
- Children
- CAPTCHAs

### Reducing training cost
If you set the __detail__ parameter for an image to __low__, the image is resized to 512 by 512 pixels and is only represented by 85 tokens regardless of its size. This will reduce the cost of training.

In [ ]:
{
    "type": "image_url",
    "image_url": {
        "url": "https://upload.wikimedia.org/wikipedia/commons/3/36/Danbo_Cheese.jpg",
        "detail": "low"
    }
}

### Preference fine-tuning
Direct Preference Optimization (DPO) fine-tuning allows you to fine-tune models based on prompts and pairs of responses. This approach enables the model to learn from human preferences, optimizing for outputs that are more likely to be favored. Note that we currently support text-only DPO fine-tuning.

#### Preparing your dataset for DPO

Each example in your dataset should contain:

- A prompt, like a user message.
- A preferred output (an ideal assistant response).
- A non-preferred output (a suboptimal assistant response).

The data should be formatted in JSONL format, with each line representing an example in the following structure:

In [ ]:
{
  "input": {
    "messages": [
      {
        "role": "user",
        "content": "Hello, can you tell me how cold San Francisco is today?"
      }
    ],
    "tools": [],
    "parallel_tool_calls": true
  },
  "preferred_output": [
    {
      "role": "assistant",
      "content": "Today in San Francisco, it is not quite cold as expected. Morning clouds will give away to sunshine, with a high near 68°F (20°C) and a low around 57°F (14°C)."
    }
  ],
  "non_preferred_output": [
    {
      "role": "assistant",
      "content": "It is not particularly cold in San Francisco today."
    }
  ]
}

#### Stacking methods: Supervised + DPO
Currently, OpenAI offers Supervised Fine-Tuning (SFT) as the default method for fine-tuning jobs. Performing SFT on your preferred responses (or a subset) before running another DPO job afterwards can significantly enhance model alignment and performance. By first fine-tuning the model on the desired responses, it can better identify correct patterns, providing a strong foundation for DPO to refine behavior.

A recommended workflow is as follows:

- Fine-tune the base model with SFT using a subset of your preferred responses. Focus on ensuring the data quality and representativeness of the tasks.
- Use the SFT fine-tuned model as the starting point, and apply DPO to adjust the model based on preference comparisons.

The __beta__ hyperparameter is a new option that is only available for DPO. It's a floating point number between __0__ and __2__ that controls how strictly the new model will adhere to its previous behavior, versus aligning with the provided preferences. A high number will be more conservative (favoring previous behavior), and a lower number will be more aggressive (favor the newly provided preferences more often).

In [ ]:
# Create a fine-tuning job with DPO
from openai import OpenAI

client = OpenAI()

job = client.fine_tuning.jobs.create(
    training_file="file-all-about-the-weather",
    model="gpt-4o-2024-08-06",
    method={
        "type": "dpo",
        "dpo": {
            "hyperparameters": {"beta": 0.1},
        },
    },
)

### Fine-tuning examples
Now that we have explored the basics of the fine-tuning API, let’s look at going through the fine-tuning lifecycle for a few different use cases.
#### Style and tone
In this example, we will explore how to build a fine-tuned model which gets the model follow specific style and tone guidance beyond what is possible with prompting alone.

To begin, we create a sample set of messages showing what the model should which in this case is misspelled words.


In [ ]:
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "Paris, as if everyone doesn't know that already."}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "Oh, just some guy named William Shakespeare. Ever heard of him?"}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "Around 384,400 kilometers. Give or take a few, like that really matters."}]}

If you want to follow along and create a fine-tuned model yourself, you will need at least 10 examples.

After getting the data that will potentially improve the model, the next step is to check if the data meets all the formatting requirements.

Now that we have the data formatted and validated, the final training step is to kick off a job to create the fine-tuned model. You can do this via the OpenAI CLI or one of our SDKs as shown below:

In [ ]:
from openai import OpenAI
client = OpenAI()

file = client.files.create(
  file=open("marv.jsonl", "rb"),
  purpose="fine-tune"
)

client.fine_tuning.jobs.create(
  training_file=file.id,
  model="gpt-4o-mini-2024-07-18"
)

Once the training job is done, you will be able to use your fine-tuned model.

#### Structured output
Another type of use case which works really well with fine-tuning is getting the model to provide structured information, in this case about sports headlines:

In [ ]:
{"messages": [{"role": "system", "content": "Given a sports headline, provide the following fields in a JSON dict, where applicable: \"player\" (full name), \"team\", \"sport\", and \"gender\"."}, {"role": "user", "content": "Sources: Colts grant RB Taylor OK to seek trade"}, {"role": "assistant", "content": "{\"player\": \"Jonathan Taylor\", \"team\": \"Colts\", \"sport\": \"football\", \"gender\": \"male\" }"}]}
{"messages": [{"role": "system", "content": "Given a sports headline, provide the following fields in a JSON dict, where applicable: \"player\" (full name), \"team\", \"sport\", and \"gender\"."}, {"role": "user", "content": "OSU 'split down middle' on starting QB battle"}, {"role": "assistant", "content": "{\"player\": null, \"team\": \"OSU\", \"sport\": \"football\", \"gender\": null }"}]}

In [ ]:
from openai import OpenAI
client = OpenAI()

file = client.files.create(
  file=open("sports-context.jsonl", "rb"),
  purpose="fine-tune"
)

client.fine_tuning.jobs.create(
  training_file=file.id,
  model="gpt-4o-mini-2024-07-18"
)

Once the training job is done, you will be able to use your fine-tuned model and make a request that looks like the following:

In [ ]:
completion = client.chat.completions.create(
    model = "ft:gpt-4o-mini:my-org:custom_suffix:id",
    messages= [
         {"role": "system", "content": "Given a sports headline, provide the following fields in a JSON dict, where applicable: player (full name), team, sport, and gender"},
        {"role": "user", "content": "Richardson wins 100m at worlds to cap comeback"}
    ]
)
print(completion.choices[0].message)

Based on the formatted training data, the response should look like the following:

In [ ]:
{
    "player": "Sha'Carri Richardson",
    "team": null,
    "sport": "track and field",
    "gender": "female"
}

### Tool calling
Fine-tuning a model with tool calling examples can allow you to:

- Get similarly formatted responses even when the full tool definition isn't present
- Get more accurate and consistent outputs
Format your examples as shown, with each line including a list of "messages" and an optional list of "tools":

In [ ]:
{
    "messages": [
        {"role": "user", "content": "What is weather in San Francisco?"},
        {
            "role": "assistant",
            "tool_calls": [
                {
                    "id": "call_id",
                    "type": "function",
                    "function": {
                        "name": "get_current_weather",
                        "arguments": "{\"location\": \"San Francisco, USA\", \"format\": \"celsius\"}"
                    }
                }
            ]
        }
    ],

    "tools": [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and country e.g. San Francisco and USA."
                        },
                        "format": {"type": "string", "enum":["celcius", "fahrenheit"]}
                    },
                    "required": ["location", "format"]
                }
            }
        }
    ]
}

If your goal is to use less tokens, some useful techniques are:

- Omit function and parameter descriptions: remove the description field from function and parameters
- Omit parameters: remove the entire properties field from the parameters object
- Omit function entirely: remove the entire function object from the functions array
  
If your goal is to maximize the correctness of the function calling output, we recommend using the same tool definitions for both training and querying the fine-tuned model.

Fine-tuning on function calling can also be used to customize the model's response to function outputs. To do this you can include a function response message and an assistant message interpreting that response:

In [ ]:
{
    "messages": [
        {"role": "user", "content": "What is the weather in San Francisco?"},
        {"role": "assistant", "tool_calls": [{"id": "call_id", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"San Francisco, USA\", \"format\": \"celsius\"}"}}]}
        {"role": "tool", "tool_call_id": "call_id", "content": "21.0"},
        {"role": "assistant", "content": "It is 21 degrees celsius in San Francisco, CA"}
    ],
    "tools": [] // same as before
}

#### Function calling
function_call and functions have been deprecated in favor of tools it is recommended to use the tools parameter instead.

The chat completions API supports function calling. Including a long list of functions in the completions API can consume a considerable number of prompt tokens and sometimes the model hallucinates or does not provide valid JSON output.

Fine-tuning a model with function calling examples can allow you to:

- Get similarly formatted responses even when the full function definition isn't present
- Get more accurate and consistent outputs

Format your examples as shown, with each line including a list of "messages" and an optional list of "functions":

In [ ]:
{
    "messages": [
        { "role": "user", "content": "What is the weather in San Francisco?" },
        {
            "role": "assistant",
            "function_call": {
                "name": "get_current_weather",
                "arguments": "{\"location\": \"San Francisco, USA\", \"format\": \"celsius\"}"
            }
        }
    ],
    "functions": [
        {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and country, eg. San Francisco, USA"
                    },
                    "format": { "type": "string", "enum": ["celsius", "fahrenheit"] }
                },
                "required": ["location", "format"]
            }
        }
    ]
}

In [ ]:
{
    "messages": [
        {"role": "user", "content": "What is the weather in San Francisco?"},
        {"role": "assistant", "function_call": {"name": "get_current_weather", "arguments": "{\"location\": \"San Francisco, USA\", \"format\": \"celsius\"}"}}
        {"role": "function", "name": "get_current_weather", "content": "21.0"},
        {"role": "assistant", "content": "It is 21 degrees celsius in San Francisco, CA"}
    ],
    "functions": [] // same as before
}

### Weights and Biases Integration
Weights and Biases (W&B) is a popular tool for tracking machine learning experiments. You can use the OpenAI integration with W&B to track your fine-tuning jobs in W&B. 

### When should I use fine-tuning vs embeddings / retrieval augmented generation?
Embeddings with retrieval is best suited for cases when you need to have a large database of documents with relevant context and information.

By default OpenAI’s models are trained to be helpful generalist assistants. Fine-tuning can be used to make a model which is narrowly focused, and exhibits specific ingrained behavior patterns. Retrieval strategies can be used to make new information available to a model by providing it with relevant context before generating its response. Retrieval strategies are not an alternative to fine-tuning and can in fact be complementary to it.

## Model distillation
Improve smaller models with distillation techniques.
Model Distillation allows you to leverage the outputs of a large model to fine-tune a smaller model, enabling it to achieve similar performance on a specific task. This process can significantly reduce both cost and latency, as smaller models are typically more efficient.

Here's how it works:

- Store high-quality outputs of a large model using the __store__ parameter in the chat completions API to store them.
- Evaluate the stored completions with both the large and the small model to establish a baseline.
- Select the stored completions that you'd like to use to for distillation and use them to fine-tune the smaller model.
- Evaluate the performance of the fine-tuned model to see how it compares to the large model.
Let's go through these steps to see how it's done.

### Store high-quality outputs of a large model

In [ ]:
#javascript
import OpenAI from "openai";
const openai = new OpenAI();

const response = await openai.chat.completions.create({
  model: "gpt-4o",
  messages: [
    { role: "system", content: "You are a corporate IT support expert." },
    { role: "user", content: "How can I hide the dock on my Mac?"},
  ],
  store: true,
  metadata: {
    role: "manager",
    department: "accounting",
    source: "homepage"
  }
});

console.log(response.choices[0]);

### Evaluate to establish a baseline
You can use your stored completions to evaluate the performance of both the larger model and a smaller model on your task to establish a baseline. This can be done using the evals product.

### Create training dataset to fine-tune smaller model
Next you can select a subset of your stored completions to use as training data for fine-tuning a smaller model like gpt-4o-mini. Filter your stored completions to those that you would like to use to train the small model, and click the "Distill" button.

### Evaluate the fine-tuned small model
When your fine-tuning job is complete, you can run evals against it to see how it stacks up against the base small and large models. You can select fine-tuned models in the Evals product to generate new completions with the fine-tuned small model.


## Evaluating model performance

Test and improve model outputs through evaluations.
Evaluations (often called evals) help you develop high-quality LLM applications by ensuring that the results you generate with a model meet accuracy criteria that you specify. 

### Understand your objective
Before you begin, you should understand the behavior you want from the model. Here are some questions you should be able to answer:

- What kind of output do I want the model to generate?
- What kinds of inputs should the model be able to handle?
- How could I tell whether or not my model output is accurate?

### Understand your objective
Before you begin, you should understand the behavior you want from the model. Here are some questions you should be able to answer:

- What kind of output do I want the model to generate?
- What kinds of inputs should the model be able to handle?
- How could I tell whether or not my model output is accurate?
  
The answers to these questions will dictate how you build your eval, and what kind of data you need to assemble before you can evaluate model performance. Let's make this more concrete with a real example.

#### Example objective - sentiment analysis of movie reviews
Let's say you are building an LLM application to perform sentiment analysis on movie reviews from IMDB. You'd like to give the model the text of a movie review, and have it output 1 if it's a positive review, and 0 if it's a negative review. For this use case, let's answer the three questions posed above:

What kind of output do I want the model to generate?

In this case, you want the model to generate just a 1 for positive reviews, and 0 for negative reviews. You don't want the model to generate anything else besides these two numbers.

What kinds of inputs should the model be able to handle?

The model should be able to process user-generated movie review text. Based on the text of the reviews, the model should be able to ascertain whether or not the overall tone of the review is positive or negative. For testing purposes, we will assume that all the reviews being tested are in English.

How could I tell whether or not my model output is accurate?

There are a few potential ways we could tell if the model output is accurate:

- We could ask another (ideally larger or more powerful) model to analyze the same review. If the other model agreed with our model's output, that could be a strong indicator of success.
- We could compare the model output to human-created output on the same review, which we could trust to be accurate. Human-labeled data could be considered ground truth, which is the reality we want to test our model outputs against.
  
The second option is likely to be most accurate - so if we have the ability to test against human-labeled data as our ground truth, that's probably the best option. Model-graded outputs can be useful when human-labeled data is not available, though.

Now that we have a firm grasp on what we want the model to generate, what kinds of inputs we want to test with, and how we'd know if the ouputs are accurate, we can move on to collecting the data we need for our eval.

#### Assemble test data
Getting good test data inputs is arguably the hardest part of building evals. That's because good test data must be representative of the kinds of inputs the model will receive in production.

Consider our movie review use case. If all our test cases were negative reviews written by the same person, it would not be a very good test dataset. We'd ideally want the percentage of positive and negative reviews to match the proportions we observe in reality, and to be written by a diverse range of authors.

When generating a test dataset, you have a few options:

- You can find an existing open source dataset that matches your use case
- You can create a synthetic dataset, either by having a human or a model generate test inputs that meet your specifications
- You can create a dataset from real production usage of your application
  
The third option is ideal, since it is likely to resemble the reality your application will face. If you are already using OpenAI, you might consider using Stored Completions for your API traffic. Setting store: true on your completions will make them show up here in the dashboard, where you can filter them to create a dataset for evals, fine-tuning, or distillation.

In [ ]:
import OpenAI from "openai";
const openai = new OpenAI();

const response = await openai.chat.completions.create({
  model: "gpt-4o",
  messages: [
    { role: "system", content: "You are a corporate IT support expert." },
    { role: "user", content: "How can I hide the dock on my Mac?"},
  ],
  store: true,
  metadata: {
    role: "manager",
    department: "accounting",
    source: "homepage"
  }
});

console.log(response.choices[0]);

For the purposes of this guide, however, we'll be using an open source dataset from Hugging Face. You can download a CSV version of the data [from here.](https://huggingface.co/datasets/stanfordnlp/imdb)